In [1]:
import scipy.io
import pandas as pd
import numpy as np
import os
import shutil
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

path_to_images = "/home/eonrrfe/Documents/Repos/Others/CarDetection/test_images/reduced_stanford_dataset/all_images"
train_images_path = "/home/eonrrfe/Documents/Repos/Others/CarDetection/test_images/reduced_stanford_dataset/train_images"
test_images_path = "/home/eonrrfe/Documents/Repos/Others/CarDetection/test_images/reduced_stanford_dataset/test_images"
path_to_annos_file = "/home/eonrrfe/Documents/Repos/Others/CarDetection/test_images/reduced_stanford_dataset/cars_train_annos.mat"
path_to_metadata_file = "/home/eonrrfe/Documents/Repos/Others/CarDetection/test_images/reduced_stanford_dataset/cars_meta.mat"

path_to_model = "/home/eonrrfe/Documents/Repos/Others/CarDetection/test_images/reduced_stanford_dataset/resnet_50_model.keras"


2024-08-27 06:23:13.137027: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-27 06:23:13.138106: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-27 06:23:13.140562: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-27 06:23:13.147542: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-27 06:23:13.158385: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [25]:
# BUILD FOLDER STRUCTURE FOR TRAINING

new_mat = scipy.io.loadmat(path_to_annos_file)
metadata_mat = scipy.io.loadmat(path_to_metadata_file)
trial = np.hstack(new_mat['annotations'])
trial_df = pd.DataFrame(trial)
trial_df['file_name'] = trial_df['fname']
trial_df['file_name'] = trial_df['file_name'].map(np.array2string)
trial_df['file_name'] = trial_df['file_name'].map(lambda x:x.lstrip("['").rstrip("']"))
trial_df['class_number'] = trial_df['class'].astype(float)
names= metadata_mat['class_names'].copy()
names_df = pd.DataFrame(names).T
names_df['class_number'] =names_df.index+1
names_df['car_name'] = names_df[0].map(lambda x:np.array2string(x).lstrip("['").rstrip("']"))
names_df = names_df.drop(columns = 0)
names_df['brand'] = names_df['car_name'].map(lambda x:x.split()[0])
names_df['model'] = names_df['car_name'].map(lambda x:x.split()[1])
names_df['both']= names_df['brand'] +  names_df['model']


names_df['both'] = names_df['both'].replace('RamC/V','RamCV')
names_df['both'] = names_df['both'].map(lambda x : x.lower())
carBrand = names_df['both'].unique()
# names_df

for i in carBrand:
  try:
    os.makedirs(f"{train_images_path}/{i}")
  except FileExistsError:
    pass

for i in carBrand:
  try:
    os.makedirs(f"{test_images_path}/{i}")
  except FileExistsError:
    pass

In [31]:
# COPY FILES ACCORDING TO TRAIN-TEST SPLIT

from sklearn.model_selection import train_test_split

merged_df_2 = trial_df.merge(names_df, left_on='class_number', right_on = 'class_number')
# merged_df_2

# test size here specifies how big percentage will be left for testing
train, test = train_test_split(merged_df_2, test_size = 0.2, random_state = 42)
# train

for b in carBrand:
  temp_df = train[train['both']==b]
  temp_df.reset_index(drop = True)
#   print(b)
  for i in list(temp_df.index.values):
    shutil.copy(f'{path_to_images}/'+temp_df['file_name'][i], f'{train_images_path}/'+b.lower()+'/'+temp_df['file_name'][i])


for b in carBrand:
  temp_df = test[test['both']==b]
  temp_df.reset_index(drop = True)
#   print(b)
  for i in list(temp_df.index.values):
    shutil.copy(f'{path_to_images}/'+temp_df['file_name'][i], f'{test_images_path}/'+b.lower()+'/'+temp_df['file_name'][i])   

In [2]:
# PREPEAR FOR TRAINING THE RESNET50 MODEL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import PIL
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = (224, 224)
batch_size = 50
transform = ImageDataGenerator(preprocessing_function= keras.applications.resnet.preprocess_input, validation_split=0.2)

# we split the training dataset further to 80-20 for training and validating

train_ds = transform.flow_from_directory(
    train_images_path,
    subset="training", seed = 42,target_size=image_size,batch_size=batch_size
)
val_ds = transform.flow_from_directory(
    train_images_path,
    subset="validation", seed = 42,target_size=image_size,batch_size=batch_size
)

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

inputs = keras.Input(shape=(224, 224, 3))
base_model = ResNet50(weights='imagenet', include_top=True, input_tensor=inputs)

last_layer = base_model.layers[-2].output 
out = Dense(units = 157, activation = 'softmax', name = 'ouput')(last_layer)
new_base_model = Model(inputs = inputs, outputs = out)

for layer in new_base_model.layers[:-25]:
  layer.trainable = False

# new_base_model.summary()

# CONTINUE:
# - build and compile Restnet50 model using this tutorial: https://github.com/ijosephp/apm_project/blob/main/Resnet50.ipynb 

Found 5264 images belonging to 157 classes.
Found 1251 images belonging to 157 classes.


In [3]:
# TRAIN THE RESNET50 MODEL AND SAVE IT LOCALLY

new_base_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss="categorical_crossentropy",
    metrics=['accuracy']
)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss',mode = 'min')

# history = new_base_model.fit_generator(generator = train_ds, epochs=50,  validation_data = val_ds,callbacks=[callback])

new_base_model.save(path_to_model) 